Попрактикуемся строить Z-критерий для биноминального распределения и получать p-value.
Дана задача, где у нас есть 50 доставок. Н0 - 0.5, Н1  - > 0.5. Ну и пусть будет 27 успешных доставок (потом будем менять числа).
Посчитаем p-value для биноминального и нормального распределения)

In [3]:
from scipy.stats import binom
from scipy.stats import norm

In [4]:
def p_value_binom(n, mu, t):
    binom_h0 = binom(n=n, p=mu)
    return 1 - binom_h0.cdf(t-1)

In [7]:
n = 50
mu = 0.5
t = 27
print(f'p-value для биномиального распределения без использования норм. аппроксимации:{p_value_binom(n, mu, t)}')

p-value для биномиального распределения без использования норм. аппроксимации:0.3359055168826828


Хоть мы и получили неадекватное p-value найдем его с помощью нормальной аппроксимации: для этого вычислим дисперсию n * $\mu$ * (1 - $\mu$)

In [10]:
def p_value_by_norm_approx(n, mu, t):
    sum_mu = n * mu
    variance = n * mu * (1 - mu)
    sigma = numpy.sqrt(variance)
    return 1 - norm(loc=sum_mu, scale=sigma).cdf(t)

In [12]:
import numpy
print(f'p-value with normal approximation: {p_value_by_norm_approx(n, mu, t)}')

p-value with normal approximation: 0.2858038224766658


Сделаем поправку на непрерывность, чтобы значения отличались на уровне погрешности:

In [15]:
def p_value_by_norm_approx_cc(n, mu, t):
    sum_mu = n * mu
    variance = n * mu * (1 - mu)
    sigma = numpy.sqrt(variance)
    return 1 - norm(loc=sum_mu, scale=sigma).cdf(t - 0.5)
print(f'p-value with normal approximation and with continuity correction: {p_value_by_norm_approx_cc(n, mu, t)}')

p-value with normal approximation and with continuity correction: 0.3356866202704363


Теперь проведем Z-test для поиска нашего истинного `p-value`

In [18]:
def p_value_z_test(sample_size, sample_mean, mu_0, variance):
    z_statistics = numpy.sqrt(sample_size) * (sample_mean - mu_0) / numpy.sqrt(variance)
    return 1 - norm().cdf(z_statistics)

In [20]:
variance = mu * (1 - mu)
p_value_z_test(n, t/n, mu, variance)

0.2858038224766656

Мы получаем точно такое же значение, что и при применении нормальной аппроксимации. Критерий Фишера мы можем применять только тогда, когда нам известна дисперсия. Или мы можем ее точно посчитать. В других же случаях мы не можем его применять)


Теперь посчитаем мощность и MDE нашего критерия, возьмем mu_factual = 0.63

In [37]:
def get_stat_power(n, mu, mu_factual, alpha=0.05):
    binom_h0 = binom(n=n, p=mu)
    binom_fact = binom(n=n, p=mu_factual)
    crit_val = binom_h0.ppf(1-alpha) + 1
    return 1 - binom_fact.cdf(crit_val)

In [43]:
n = 50
mu = 0.5
mu_factual = 0.63
get_stat_power(n, mu, mu_factual)

0.3894766386973454

Видим, что мощность ниже 80%, поэтому можно посчитать, какое количество экспеиментов надо провести, чтобы для фактического mu этот критерий был мощным

In [46]:
def power_min_n(n):
    power = get_stat_power(n, mu=0.5, mu_factual=0.63)
    while power <= 0.8:
        n += 1
        power = get_stat_power(n,mu=0.5, mu_factual=0.63)
    return n

In [48]:
power_min_n(n)

106

In [50]:
def MDE(n, mu0, alpha=0.05, min_power=0.8):
    delta_grid = numpy.linspace(0, 1 - mu0, 200)
    power = get_stat_power(n, mu0, mu0 + delta_grid)
    delta = delta_grid[power >= 0.8]
    return delta[0]

In [58]:
MDE(n=50, mu0=0.5)

0.20603015075376885